# Fine-tuning GPT-2

In [ ]:
# paths to dataset files stored in Google Drive
DATA_DIR = '/content/drive/MyDrive/KeepCoding/Bootcamp_AI/8.LLMs/Assignment/'

# instructed dataset
PATH_HYROX_JSON_INSTRUCTED = f'{DATA_DIR}/hyrox_json_instructed.jsonl'
PATH_HYROX_JSON_PROMPT_INSTRUCTED = f'{DATA_DIR}/hyrox_json_prompt_instructed.jsonl'
PATH_HYROX_SPLIT_DATASET = f'{DATA_DIR}/hyrox_split_dataset'
PATH_HYROX_TOKENIZER_GPT2 = f'{DATA_DIR}/hyrox_gpt2_tokenizer'
PATH_HYROX_MODEL_GPT2 = f'{DATA_DIR}/hyrox_gpt2_model'

In [ ]:
!pip install accelerate bitsandbytes datasets transformers trl -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
from datasets import Dataset
from datasets import DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import GenerationConfig, pipeline, set_seed
from transformers import TrainingArguments
from trl import SFTTrainer


In [ ]:
# load the data into a pandas DataFrame
hyrox_df = pd.read_json(PATH_HYROX_JSON_INSTRUCTED, lines=True)

# convert it to a Dataset object
hyrox_dataset = Dataset.from_pandas(hyrox_df)

# shape
hyrox_dataset.shape

(5256, 3)

In [ ]:
hyrox_dataset[0]

{'input': 'Generate performance feedback based on prediction and true time',
 'context': 'gender: 0, age: 16-24, total_time: 3407, predicted: 3010, residual: 397, cluster: 4',
 'response': 'You performed slower than expected by more than 3 minutes. Compared to your cluster (#4.0), you may want to improve: 1000m Row, 200m Farmer Carry, 100m Sandbag Lunges, 100 Wall Balls, work_run_time, roxzone_time.'}

In [ ]:
hyrox_dataset_train_test = hyrox_dataset.train_test_split(test_size=0.2)

In [ ]:
hyrox_dataset_train_test

DatasetDict({
    train: Dataset({
        features: ['input', 'context', 'response'],
        num_rows: 4204
    })
    test: Dataset({
        features: ['input', 'context', 'response'],
        num_rows: 1052
    })
})

In [ ]:
hyrox_dataset_val_test = hyrox_dataset_train_test['test'].train_test_split(test_size=0.5)

In [ ]:
hyrox_dataset_val_test

DatasetDict({
    train: Dataset({
        features: ['input', 'context', 'response'],
        num_rows: 526
    })
    test: Dataset({
        features: ['input', 'context', 'response'],
        num_rows: 526
    })
})

In [ ]:
split_hyrox_dataset = DatasetDict({
    'train' : hyrox_dataset_train_test['train'],
    'val' : hyrox_dataset_val_test['train'],
    'test' : hyrox_dataset_val_test['test']
})

In [ ]:
split_hyrox_dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'context', 'response'],
        num_rows: 4204
    })
    val: Dataset({
        features: ['input', 'context', 'response'],
        num_rows: 526
    })
    test: Dataset({
        features: ['input', 'context', 'response'],
        num_rows: 526
    })
})

# Training Prompt Creation

Two promts, one for training and other for inference which doesn't include the response.

In [ ]:
HYROX_TRAINING_PROMPT_TEMPLATE = """\
{bos_token}### Instruction:
{system_message}

### Input:
{input}

### Context:
{context}

### Response:
{response}{eos_token}
"""

HYROX_INFERENCE_PROMPT_TEMPLATE = """\
{bos_token}### Instruction:
{system_message}

### Input:
{input}

### Context:
{context}

### Response:{eos_token}
"""

# System Message

In [ ]:
SYSTEM_MESSAGE = "You are a performance analysis assistant for Hyrox athletes. Your job is to analyze performance data provided by atheles once they complete a Hyrox race and generate personalized, insightful feedback that helps the athlete improve."

In [ ]:
def create_hyrox_prompt(sample):
  full_prompt = HYROX_TRAINING_PROMPT_TEMPLATE.format(
      bos_token = '<|startoftext|>',
      eos_token = '<|endoftext|>',
      system_message = SYSTEM_MESSAGE,
      input = sample['input'],
      context = sample['context'],
      response = sample['response']
  )

  return {"text" : full_prompt}

In [ ]:
def create_hyrox_prompt_and_response(sample):
  full_prompt = HYROX_INFERENCE_PROMPT_TEMPLATE.format(
      bos_token = '<|startoftext|>',
      eos_token = '<|endoftext|>',
      system_message = SYSTEM_MESSAGE,
      input = sample['input'],
      context = sample['context']
  )

  ground_truth = sample['response']

  return {"full_prompt" : full_prompt, "ground_truth" : ground_truth}

In [ ]:
split_hyrox_dataset['train'][0]

{'input': 'Generate performance feedback based on prediction and true time',
 'context': 'gender: 1, age: 25-29, total_time: 4909, predicted: 5063, residual: -154, cluster: 0',
 'response': 'You performed close to expected levels. Compared to your cluster (#0.0), you may want to improve: 1000m Row, work_run_time.'}

In [ ]:
create_hyrox_prompt(split_hyrox_dataset['train'][0])

{'text': '<|startoftext|>### Instruction:\nYou are a performance analysis assistant for Hyrox athletes. Your job is to analyze performance data provided by atheles once they complete a Hyrox race and generate personalized, insightful feedback that helps the athlete improve.\n\n### Input:\nGenerate performance feedback based on prediction and true time\n\n### Context:\ngender: 1, age: 25-29, total_time: 4909, predicted: 5063, residual: -154, cluster: 0\n\n### Response:\nYou performed close to expected levels. Compared to your cluster (#0.0), you may want to improve: 1000m Row, work_run_time.<|endoftext|>\n'}

In [ ]:
# apply it to whole dataset
split_hyrox_dataset = split_hyrox_dataset.map(create_hyrox_prompt)

Map:   0%|          | 0/4204 [00:00<?, ? examples/s]

Map:   0%|          | 0/526 [00:00<?, ? examples/s]

Map:   0%|          | 0/526 [00:00<?, ? examples/s]

In [ ]:
split_hyrox_dataset['train'][0]

{'input': 'Generate performance feedback based on prediction and true time',
 'context': 'gender: 1, age: 25-29, total_time: 4909, predicted: 5063, residual: -154, cluster: 0',
 'response': 'You performed close to expected levels. Compared to your cluster (#0.0), you may want to improve: 1000m Row, work_run_time.',
 'text': '<|startoftext|>### Instruction:\nYou are a performance analysis assistant for Hyrox athletes. Your job is to analyze performance data provided by atheles once they complete a Hyrox race and generate personalized, insightful feedback that helps the athlete improve.\n\n### Input:\nGenerate performance feedback based on prediction and true time\n\n### Context:\ngender: 1, age: 25-29, total_time: 4909, predicted: 5063, residual: -154, cluster: 0\n\n### Response:\nYou performed close to expected levels. Compared to your cluster (#0.0), you may want to improve: 1000m Row, work_run_time.<|endoftext|>\n'}

In [ ]:
# save the split dataset
split_hyrox_dataset.save_to_disk(PATH_HYROX_SPLIT_DATASET)

Saving the dataset (0/1 shards):   0%|          | 0/4204 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/526 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/526 [00:00<?, ? examples/s]

<|startoftext|>

### Instruction:

You are a performance analysis assistant for Hyrox athletes. Your job is to analyze performance data provided by atheles once they complete a Hyrox race and generate personalized, insightful feedback that helps the athlete improve.

### Input:

Generate performance feedback based on prediction and true time

### Context:
gender: 0, age: 35-39, total_time: 5438, predicted: 5780, residual: -342, cluster: None\n\n

### Response:\
You outperformed the model's prediction significantly — great job! You are performing in line with your cluster's average strengths.

<|endoftext|>

# Load the model to tune and preprocess it

In [ ]:
model_id = "gpt2"
# model_id = "mistralai/Mistral-7B-Instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_id)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

Training pipeline

In [ ]:
# init the text generation pipeline with model and tokenizer
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# set the seed
set_seed(42)

def generate_sample(sample):
  # Prepare the prompt generation, including the question and answer expected.
  prompt_package = create_hyrox_prompt_and_response(sample)

  # Config for text generation, setting params
  generation_config = GenerationConfig(
      max_new_tokens=50,
      do_sample=True,
      top_k=50,
      temperature=1e-4,
      eos_token_id=model.config.eos_token_id,
  )

  # Generate text based on the prompt and config
  generation = generator(prompt_package["full_prompt"], generation_config=generation_config)
  print("---------------")
  print("Question:")
  print(sample["input"])
  print("---------------")
  print("Dataset Response")
  # Imprime la respuesta esperada para comparación.
  print(prompt_package["ground_truth"])
  print("---------------")
  print("Model Response:")
  # Imprime la respuesta del modelo, eliminando el texto del prompt para mostrar solo el texto generado nuevo.
  print(generation[0]["generated_text"].replace(prompt_package["full_prompt"], ""))


Device set to use cpu


In [ ]:
generate_sample(split_hyrox_dataset["test"][20])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Question:
Generate performance feedback based on prediction and true time
---------------
Dataset Response
You outperformed the model’s prediction significantly — great job! Compared to your cluster (#5.0), you may want to improve: 50m Sled Push, 50m Sled Pull, 80m Burpee Broad Jump, 1000m Row, 200m Farmer Carry, 100m Sandbag Lunges, work_run_time.
---------------
Model Response:
The following is a list of all the games that have been released for the Xbox One.

Xbox One X

Xbox One X

Xbox One X

Xbox One X

Xbox One X

Xbox One X




The model is unabled to perform the task.

In [ ]:
training_args = TrainingArguments(
 per_device_train_batch_size=4,
 gradient_accumulation_steps=4,
 gradient_checkpointing =True,
 max_grad_norm= 0.3,
 #num_train_epochs=2,
 max_steps=100,
 learning_rate=2e-4,
 save_total_limit=3,
 logging_steps=10,
 output_dir=PATH_HYROX_MODEL_GPT2,
 optim="paged_adamw_32bit",
 lr_scheduler_type="cosine",
 #evaluation_strategy="steps",
 eval_steps=50,
 warmup_ratio=0.05,
    report_to="none",
)

In [ ]:
trainer = SFTTrainer(
 model,
 train_dataset=split_hyrox_dataset['train'],
 eval_dataset=split_hyrox_dataset['val'],
 #tokenizer=gpt2_tokenizer,
 args=training_args
)

Converting train dataset to ChatML:   0%|          | 0/4204 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/4204 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/4204 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/4204 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/526 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/526 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/526 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/526 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
10,0.232600
20,0.220500
30,0.218400
40,0.211900
50,0.213600
60,0.213800
70,0.213400
80,0.209000
90,0.210300
100,0.212100


TrainOutput(global_step=100, training_loss=0.21554532766342163, metrics={'train_runtime': 117.954, 'train_samples_per_second': 13.565, 'train_steps_per_second': 0.848, 'total_flos': 135300280320000.0, 'train_loss': 0.21554532766342163})

In [ ]:
trainer.save_model()

In [ ]:
tokenizer.save_pretrained(PATH_HYROX_TOKENIZER_GPT2)

('/content/drive/MyDrive/KeepCoding/Bootcamp_AI/8.LLMs/Assignment//hyrox_gpt2_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/KeepCoding/Bootcamp_AI/8.LLMs/Assignment//hyrox_gpt2_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/KeepCoding/Bootcamp_AI/8.LLMs/Assignment//hyrox_gpt2_tokenizer/vocab.json',
 '/content/drive/MyDrive/KeepCoding/Bootcamp_AI/8.LLMs/Assignment//hyrox_gpt2_tokenizer/merges.txt',
 '/content/drive/MyDrive/KeepCoding/Bootcamp_AI/8.LLMs/Assignment//hyrox_gpt2_tokenizer/added_tokens.json',
 '/content/drive/MyDrive/KeepCoding/Bootcamp_AI/8.LLMs/Assignment//hyrox_gpt2_tokenizer/tokenizer.json')

# Test the model

In [ ]:
hyrox_gpt2_tokenizer = AutoTokenizer.from_pretrained(PATH_HYROX_TOKENIZER_GPT2)

In [ ]:
hyrox_gpt2_model = AutoModelForCausalLM.from_pretrained(PATH_HYROX_MODEL_GPT2)

In [ ]:
generator = pipeline('text-generation', model=hyrox_gpt2_model, tokenizer=hyrox_gpt2_tokenizer)

Device set to use cpu


In [ ]:
for i in range(20):
    generate_sample(split_hyrox_dataset["test"][i])
    print('--------------------------------------------------------')

`generation_config` default values have been modified to match model-specific defaults: {'bos_token_id': 50256}. If this is not desired, please set these values explicitly.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Question:
Generate performance feedback based on prediction and true time
---------------
Dataset Response
You performed slower than expected by more than 3 minutes. Compared to your cluster (#0.0), you may want to improve: 1000m Row.
---------------
Model Response:
You performed slower than expected by more than 3 minutes. Compared to your cluster (#0.0), you may want to improve: 1000m Ski, 50m Sled Push, 50m Sled Pull, 80m Burpee Broad Jump, 1000
--------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Question:
Generate performance feedback based on prediction and true time
---------------
Dataset Response
You performed close to expected levels. Compared to your cluster (#3.0), you may want to improve: 1000m Ski, 50m Sled Push, 50m Sled Pull, 80m Burpee Broad Jump, 1000m Row, 200m Farmer Carry, 100m Sandbag Lunges, 100 Wall Balls, work_run_time, roxzone_time.
---------------
Model Response:
You outperformed the model’s prediction significantly — great job! Compared to your cluster (#3.0), you may want to improve: 1000m Row, 200m Farmer Carry, 100 Wall Balls, work_run_time, rox
--------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

---------------
Question:
Generate performance feedback based on prediction and true time
---------------
Dataset Response
You outperformed the model’s prediction significantly — great job! You are performing in line with your cluster’s average strengths.
---------------
Model Response:
You outperformed the model’s prediction significantly — great job! You are performing in line with your cluster’s average strengths.
--------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Question:
Generate performance feedback based on prediction and true time
---------------
Dataset Response
You performed close to expected levels. You are performing in line with your cluster’s average strengths.
---------------
Model Response:
You outperformed the model’s prediction significantly — great job! You are performing in line with your cluster’s average strengths.
--------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Question:
Generate performance feedback based on prediction and true time
---------------
Dataset Response
You performed slower than expected by more than 3 minutes. You are performing in line with your cluster’s average strengths.
---------------
Model Response:
You performed slower than expected by more than 3 minutes. You are performing in line with your cluster’s average strengths.
--------------------------------------------------------
---------------
Question:
Generate performance feedback based on prediction and true time
---------------
Dataset Response
You performed close to expected levels. Compared to your cluster (#1.0), you may want to improve: 50m Sled Pull.
---------------
Model Response:
You outperformed the model’s prediction significantly — great job! Compared to your cluster (#1.0), you may want to improve: 1000m Row, 200m Farmer Carry, 100 Wall Balls, work_run_time, rox
--------------------------------------------------------
